# How-To, First Steps

## Load package and create basic options

In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath('../'))

from robustfpm.finance import make_option

Let's start with some basic options. Remember that 1D PutOnMax is just Put (the same goes for CallOnMax)

In [2]:
option1 = make_option(option_type='putonmax', strike=90)
option2 = make_option(option_type='putonmax', strike=80, payoff_dates = 5)
option3 = make_option(option_type='callonmax', strike=90, payoff_dates = [3,5])
option4 = make_option(option_type='put2call1') # American Put2Call1 option

Now let's see how these options got instantiated

In [3]:
print('Option 1: {}'.format(option1))
print('Option 2: {}'.format(option2))
print('Option 3: {}'.format(option3))
print('Option 4: {}'.format(option4))

Option 1: <robustfpm.finance.derivatives.AmericanOption object at 0x10512e550>
Option 2: <robustfpm.finance.derivatives.EuropeanOption object at 0x10512e790>
Option 3: <robustfpm.finance.derivatives.BermudanOption object at 0x113bb3ee0>
Option 4: <robustfpm.finance.derivatives.AmericanOption object at 0x113bb3f10>


In [4]:
print('Option 1 payoff at x = 80, 90, and 100, t = 3: {}'.format(option1.payoff([[80], [90], [100]], 3)))
print('Option 2 payoff at x = 100, t = 3: {}'.format(option2.payoff(100, 3)))
print('Option 3 payoff at x = 100, t = 3: {}'.format(option3.payoff(100, 3)))
print('Option 3 payoff at x = 100, t = 4: {}'.format(option3.payoff(100, 4)))

Option 1 payoff at x = 80, 90, and 100, t = 3: [10.  0.  0.]
Option 2 payoff at x = 100, t = 3: [-inf]
Option 3 payoff at x = 100, t = 3: [10.]
Option 3 payoff at x = 100, t = 4: [-inf]


## Creating solver and solving some problems

Now let's create some basic 1D Problem with Rectangular multiplicative dynamics and no trading constraints. For that, we need module `robustfpm.pricing`

In [5]:
from robustfpm.pricing import *
import numpy as np
pm1 = Problem(starting_price=np.array(100), 
            price_dynamics=ConstantDynamics(support=RectangularHandler([.9, 1.1]), type='mult'),
            trading_constraints=NoConstraints, option=option1, 
            lattice=Lattice(delta=[1]), 
            time_horizon=5)

Now we create solver with some parameters.

Most of the time, there is no point in tweaking *all* of these parameters, only some, namely `enable_timer` and `iter_tick`

In [6]:
opts = {'convex_hull_filter': 'qhull', 'convex_hull_prune_fail_count': 0,
        'convex_hull_prune_success_count':0,'convex_hull_prune_corner_n': 3,'convex_hull_prune_seed': 0} 
solver = ConvhullSolver(enable_timer=True, pricer_options=opts, ignore_warnings=True, iter_tick=50)

Now we solve it and see the result $V_0(x_0)$

In [7]:
sol1 = solver.solve(pm1)
# the solution is simply a dictionary
print('Value: {0}'.format(sol1['Vf'][0][0]))

Precalculating points for value function evaluation: 0.0956 sec (CPU 0.0956 sec)
Computing value function in the last point: 0.0000 sec (CPU 0.0000 sec)
t = 4
iter = 54/90 (60.00%)
iter = 89/90 (98.89%)
t = 3
iter = 54/67 (80.60%)
t = 2
t = 1
iter = 18/23 (78.26%)
t = 0
Computing value function in intermediate points in time: 0.4487 sec (CPU 0.4487 sec)
Solving the problem: 0.5453 sec (CPU 0.5453 sec)
Value: 5.327786420219619


Now let's also calculate and check precision

In [8]:
sol1 = solver.solve(pm1, calc_precision = True, do_precalc=False)
# the solution is simply a dictionary
print('Value: {0}'.format(sol1['Vf'][0][0]))

Precalculating points for value function evaluation: 0.0871 sec (CPU 0.0871 sec)
Computing A^* and Lipschitz constants for constraints: 0.0003 sec (CPU 0.0003 sec)
Calculating precision: 0.0004 sec (CPU 0.0004 sec)
Computing value function in the last point: 0.0000 sec (CPU 0.0000 sec)
t = 4
iter = 81/90 (90.00%)
t = 3
iter = 50/67 (74.63%)
iter = 55/67 (82.09%)
t = 2
iter = 7/45 (15.56%)
t = 1
t = 0
Computing value function in intermediate points in time: 0.4240 sec (CPU 0.4240 sec)
Solving the problem: 0.5130 sec (CPU 0.5130 sec)
Value: 5.327786420219619


In [9]:
print('Precision: {0}'.format(sol1['precision']))

Precision: [5.85182876 4.00129318 2.49774194 1.33225806 0.5        0.        ]


Let's play around and change Lattice step, especially since the error is too big.

In [10]:
pm1.lattice = Lattice(delta=[.1])
solver
sol2 = solver.solve(pm1, calc_precision = True)
print('Value: {0}'.format(sol2['Vf'][0][0]))
print('Precision: {0}'.format(sol2['precision']))

Precalculating points for value function evaluation: 0.6532 sec (CPU 0.6532 sec)
Computing A^* and Lipschitz constants for constraints: 0.0005 sec (CPU 0.0005 sec)
Calculating precision: 0.0006 sec (CPU 0.0006 sec)
Computing value function in the last point: 0.0001 sec (CPU 0.0001 sec)
t = 4
iter = 7/818 (0.86%)
iter = 56/818 (6.85%)
iter = 86/818 (10.51%)
iter = 90/818 (11.00%)
iter = 115/818 (14.06%)
iter = 240/818 (29.34%)
iter = 270/818 (33.01%)
iter = 390/818 (47.68%)
iter = 414/818 (50.61%)
iter = 437/818 (53.42%)
iter = 453/818 (55.38%)
iter = 519/818 (63.45%)
iter = 661/818 (80.81%)
iter = 670/818 (81.91%)
iter = 689/818 (84.23%)
iter = 789/818 (96.45%)
iter = 814/818 (99.51%)
t = 3
iter = 99/609 (16.26%)
iter = 118/609 (19.38%)
iter = 214/609 (35.14%)
iter = 227/609 (37.27%)
iter = 265/609 (43.51%)
iter = 272/609 (44.66%)
iter = 283/609 (46.47%)
iter = 314/609 (51.56%)
iter = 335/609 (55.01%)
iter = 344/609 (56.49%)
iter = 441/609 (72.41%)
iter = 483/609 (79.31%)
iter = 569/60

And what if we go to the extreme?

In [15]:
pm1.lattice = Lattice(delta=[.01])
solver.reset() # otherwise, we get old precision
precision = solver.get_precision(pm1)
print('Precision: {0}'.format(precision))

Precalculating points for value function evaluation: 9.2307 sec (CPU 9.2305 sec)
Computing A^* and Lipschitz constants for constraints: 0.0010 sec (CPU 0.0010 sec)
Calculating precision: 0.0002 sec (CPU 0.0002 sec)
Precision: [0.05427884 0.03741037 0.02364124 0.01286389 0.005      0.        ]


In [14]:
solver.iter_tick = 1000
sol3 = solver.solve(pm1)
print('Value: {0}'.format(sol3['Vf'][0][0]))

Precalculating points for value function evaluation: 9.1385 sec (CPU 9.1383 sec)
Computing value function in the last point: 0.0008 sec (CPU 0.0008 sec)
t = 4
iter = 3309/8089 (40.91%)
iter = 5327/8089 (65.85%)
iter = 6215/8089 (76.83%)
t = 3
iter = 80/6027 (1.33%)
iter = 81/6027 (1.34%)
iter = 206/6027 (3.42%)
iter = 541/6027 (8.98%)
iter = 1975/6027 (32.77%)
iter = 2422/6027 (40.19%)
iter = 3981/6027 (66.05%)
t = 2
iter = 89/4005 (2.22%)
iter = 325/4005 (8.11%)
iter = 1080/4005 (26.97%)
iter = 1683/4005 (42.02%)
iter = 2608/4005 (65.12%)
iter = 3230/4005 (80.65%)
t = 1
iter = 479/2003 (23.91%)
iter = 700/2003 (34.95%)
t = 0
Computing value function in intermediate points in time: 58.8474 sec (CPU 58.8464 sec)
Solving the problem: 1 min 7.9880 sec (CPU 1 min 7.9869 sec)
Value: 4.322815140670112


A significant gain in precision — but not so much in the Value...

That's because we give a *guaranteed* estimate for precision that depends on grid and assumes that the $v_t$ is the *worst*.

Let's try 2D Problem with another option and *additive* dynamics.

In [17]:
pm2 = Problem(starting_price=np.array([91,90]), price_dynamics=ConstantDynamics(support=RectangularHandler([[-1, 1],[-.75, 1]]), type='add'),
             trading_constraints=IdenticalMap(RealSpaceHandler()),
             option=option4,
             lattice=Lattice(delta=[.1,.1]), time_horizon=5)
solver.iter_tick = 500
sol5 = solver.solve(pm2, calc_precision = True)
print('Value: {0}'.format(sol5['Vf'][0][0]))
print('Precision: {0}'.format(sol5['precision']))

Precalculating points for value function evaluation: 0.8709 sec (CPU 0.8709 sec)
Computing A^* and Lipschitz constants for constraints: 0.0009 sec (CPU 0.0009 sec)
Calculating precision: 0.0001 sec (CPU 0.0001 sec)
Computing value function in the last point: 0.0006 sec (CPU 0.0006 sec)
t = 4
iter = 612/5913 (10.35%)
iter = 782/5913 (13.23%)
iter = 1336/5913 (22.59%)
iter = 1858/5913 (31.42%)
iter = 2483/5913 (41.99%)
iter = 2716/5913 (45.93%)
iter = 4390/5913 (74.24%)
iter = 5052/5913 (85.44%)
iter = 5624/5913 (95.11%)
t = 3
iter = 479/3355 (14.28%)
iter = 595/3355 (17.73%)
iter = 1644/3355 (49.00%)
iter = 2018/3355 (60.15%)
iter = 2183/3355 (65.07%)
iter = 2259/3355 (67.33%)
iter = 2297/3355 (68.46%)
iter = 3035/3355 (90.46%)
t = 2
iter = 585/1517 (38.56%)
iter = 772/1517 (50.89%)
iter = 804/1517 (53.00%)
iter = 1119/1517 (73.76%)
iter = 1356/1517 (89.39%)
t = 1
iter = 377/399 (94.49%)
t = 0
Computing value function in intermediate points in time: 37.3468 sec (CPU 37.3462 sec)
Solving

In [ ]:
pm3 = Problem(starting_price=np.array([91,90]), price_dynamics=ConstantDynamics(support=RectangularHandler([[-1, 1],[-.75, 1]]), type='add'),
             trading_constraints=IdenticalMap(RealSpaceHandler()),
             option=option4,
             lattice=Lattice(delta=[.1,.1]), time_horizon=5)
solver.iter_tick = 500
sol6 = solver.solve(pm3, calc_precision = True)
print('Value: {0}'.format(sol6['Vf'][0][0]))
print('Precision: {0}'.format(sol6['precision']))

In [ ]:
pm4 = Problem(starting_price=np.array([91,90]), price_dynamics=ConstantDynamics(support=RectangularHandler([[-1, 1],[-.75, 1]]), type='add'),
             trading_constraints=LongOnlyConstraints,
             option=option4,
             lattice=Lattice(delta=[.1,.1]), time_horizon=5)
solver.iter_tick = 500
sol7 = solver.solve(pm4, calc_precision = True)
print('Value: {0}'.format(sol7['Vf'][0][0]))
print('Precision: {0}'.format(sol7['precision']))